In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./"))

# Any results you write to the current directory are saved as output.

['.cache', '.keras', 'patient_with_reports.csv', '.config', 'MIMIC Explore V3.ipynb', 'mimic-300D-3.5M.bin', 'anaconda3', 'ChinaSet_AllFiles.zip', 'IR', 'final.csv', '.jupyter', 'text_cleaned.csv', 'model3.pth', 'PatY.pickle', 'model-mimic.bin.wv.vectors.npy', 'model-mimic.bin.trainables.syn1neg.npy', '.python_history', 'nltk_data', 'MIMIC Explore Edition Notebook.ipynb', 'MIMIC Explore V2.ipynb', '.profile', '.ipython', 'list_of_report.csv', 'model-mimic.bin', '.bashrc', 'model1.pth', 'without_prepro_model1.pth', 'PatientCombo.csv', 'FINAL INTEGRATED MODEL .ipynb', 'NewPatReportMatrix.pickle', '.bash_history', 'PatComboArr.pickle', 'adm_disease_probablity.csv', '.nano', 'model4.pth', 'text_cleaned.csv.zip', 'PatientCombo.csvC', 'kernel.ipynb', 'weights-GRU.best.hdf5', '.local', '.bash_logout', 'alexmodel1.pth', '.torch', 'ChinaSet_AllFiles', 'PatComboShuffle.pickle', '.ipynb_checkpoints', 'kernel (6).ipynb', 'Generator_keras.ipynb', 'ChestModel.ipynb', 'data', 'MIMIC EXPLORE V4.ipynb'

In [2]:
import zipfile
zip_ref = zipfile.ZipFile("text_cleaned.csv.zip", 'r')
zip_ref.extractall()
zip_ref.close()

In [2]:
df=pd.read_csv('./text_cleaned.csv')
df.head(5)

,TEXT,word_count
0,admission date discharge date service addendum...,60
1,admission date discharge date date birth sex f...,1034
2,admission date discharge date service cardioth...,855
3,admission date discharge date service medicine...,1474
4,admission date discharge date date birth sex s...,1259


In [3]:
train_file=df['TEXT'].values
train_file.shape

(2083180,)

In [4]:
train_file[30]

'admission date discharge date date birth sex f service medicine history present illness female extensive history alcohol abuse class b child cirrhosis abstinent alcohol since saw primary care physician complaints history general malaise abdominal pain found white count notified three days thereafter told go emergency room presented emergency department abdominal pain hypotensive baseline systolic pressure responsive intravenous fluids started dopamine sent unit presentation white count total bilirubin urinalysis positive escherichia coli subsequent abdomen ultrasound showed ascites right upper quadrant ultrasound showed gallbladder inflammation consistent cholecystitis started ceftriaxone vancomycin flagyl endoscopic retrograde cholangiopancreatography ward name common bile duct stent transient elevation amylase lipase status post stent falling time transfer weaned pressors followed gastroenterology general surgery evening transfer medicine service tolerating solids without nausea vom

In [5]:
from nltk.tokenize import word_tokenize 

def data_list(big_list):
    data_list=[]
    for i in range(len(big_list)):
        curr_string=str(big_list[i])
        curr_string=curr_string.lower()
        word_tokens = word_tokenize(curr_string)
        data_list.append(word_tokens) 
    return data_list

In [5]:
from gensim.models import Word2Vec

# train model
new_list=data_list(train_file)
print("Strating")
model = Word2Vec(new_list,size=300, min_count=1)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
print(len(words))
# access vector for one word
print(model['admission'])
# save model
model.save('mimic-300D-3.5M.bin')
# load model
new_model = Word2Vec.load('mimic-300D-3.5M.bin')
print(new_model)

Strating
Word2Vec(vocab=347113, size=300, alpha=0.025)
347113
[-1.3602095  -3.9968271  -3.569002   -0.09831091 -3.376415   -2.3465135
 -1.8254884   0.37712762 -1.8530942  -4.0837393   2.7218137   1.6821749
 -1.573987    0.5103347   0.03972801  2.8294103  -0.3880204   0.12649122
 -1.6709422  -3.1543174  -2.5914185   2.8972301  -1.9932123   0.77022225
 -2.8854449  -3.3123333  -0.156261   -0.2994684  -3.2708588   3.025939
 -1.1029148   0.25915602  3.4952962  -0.579748    0.4410795   3.2813609
 -0.7046842  -0.24219711  1.0056903  -2.2699847   2.2628531   1.5454738
 -0.12766461 -2.430933    0.24728878 -5.9576607   2.469833   -0.74131554
  0.8932214   1.8288503   0.5567043   0.42554742  0.64279336  3.0933473
  0.43160138  0.01678712  3.358536   -1.8507663   1.0828675  -2.091064
 -0.46389338  2.9369032   1.6309198   0.7936424  -2.9809077   0.67704165
  3.29595    -4.076585   -3.6533484  -0.58955437 -2.6442525  -3.6278021
  0.8872541  -1.9872634   0.06706046 -1.9831231  -0.36076048  1.2086655


/home/nachiket/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


Word2Vec(vocab=347113, size=300, alpha=0.025)


In [8]:
from gensim.models import Doc2Vec,TaggedDocument

# train model
new_list=data_list(train_file)
print("Strating")


tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(new_list)]

max_epochs = 100
vec_size = 300
alpha = 0.05

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.0003,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("Doc2Vec-300D-mimic.model")
print("Model Saved")

Strating


NameError: name 'TaggedDocument' is not defined

In [13]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/nachiket/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True